In [92]:
import csv
from glob import glob
import json
import os
import random

import pandas as pd

In [77]:
random.seed=42

In [90]:
dwug_path = os.path.expanduser("~/PycharmProjects/gloss-annotator/wugs/dwug_en/")

In [107]:
label_data = []

CLUSTER_NUMBER_COLUMN = 'cluster'
EXAMPLE_COLUMN = 'context'
WORD_COLUMN = 'word'

for word_path in glob("/home/m/Downloads/glmlarge_wordnet_norml1/glmlarge_wordnet_glosses/glosses_uses_norm_l1/dwug_en/*"):
    word = os.path.split(word_path)[-1]

    word_uses = pd.read_csv(os.path.join(dwug_path, "data", word, "uses.csv"), sep="\t", quoting=csv.QUOTE_NONE)

    clusters_dir = os.path.join(dwug_path, "clusters/opt")
    if not os.path.exists(clusters_dir):  # no opt in RuDSI data
        clusters_dir = os.path.join(dwug_path, "clusters")
    word_clusters = pd.read_csv(os.path.join(clusters_dir, f"{word}.csv"), sep="\t", quoting=csv.QUOTE_NONE)
    word_clusters[CLUSTER_NUMBER_COLUMN] = word_clusters[CLUSTER_NUMBER_COLUMN].astype(int)

    
    this_word = word_uses.join(word_clusters.set_index("identifier"), on="identifier")
    #print(this_word.head())
    
    clusters_and_definitions = pd.read_csv(os.path.join(word_path, "cluster_gloss.tsv"), sep="\t")
    clusters_and_definitions["cluster"] = clusters_and_definitions.cluster.astype(int)
    clusters_and_definitions = clusters_and_definitions[clusters_and_definitions.cluster!=-1]
    if clusters_and_definitions.shape[0] > 1:
        
    
        definitions, contexts_list = [], []
        for row in clusters_and_definitions.iterrows():
            cluster_number, definition = row[1]
            
            
            this_cluster = this_word[this_word[CLUSTER_NUMBER_COLUMN] == cluster_number]
            
            contexts = []
    
            for row in this_cluster.iterrows():
                start, end = row[1]["indexes_target_token"].split(":")
                start, end = int(start), int(end)
                example = row[1][EXAMPLE_COLUMN]
                contexts.append(f"{example[:start]}<b>{example[start:end]}</b>{example[end:]}")

            if not contexts: 
                print(word)
                print(cluster_number)
            try:
                contexts_list.append('<br>'.join(set(random.choices(contexts, k=5))))
            except IndexError:
                print(contexts)
                raise IndexError
            definitions.append(definition)
        
        for definition in definitions:
            cluster_data = {}
            cluster_data["data"] = {"my_text": f"{word.upper()}: {definition}"}
            cluster_data["data"]["variants"] = [{"html": ctx} for ctx in contexts_list]
            label_data.append(cluster_data)

In [108]:
with open(os.path.expanduser("~/PycharmProjects/label-studio.json"), "w") as f:
    json.dump(label_data, f)

In [109]:
label_data = []
lang = "de"
dwug_path = os.path.expanduser(f"~/PycharmProjects/gloss-annotator/wugs/dwug_{lang}/")
CLUSTER_NUMBER_COLUMN = 'cluster'
EXAMPLE_COLUMN = 'context'
WORD_COLUMN = 'word'

for word_path in glob(f"/home/m/Downloads/glmlarge_wordnet_norml1/glmlarge_wordnet_glosses/glosses_uses_norm_l1/dwug_{lang}/*"):
    word = os.path.split(word_path)[-1]

    word_uses = pd.read_csv(os.path.join(dwug_path, "data", word, "uses.csv"), sep="\t", quoting=csv.QUOTE_NONE)

    clusters_dir = os.path.join(dwug_path, "clusters/opt")
    if not os.path.exists(clusters_dir):  # no opt in RuDSI data
        clusters_dir = os.path.join(dwug_path, "clusters")
    word_clusters = pd.read_csv(os.path.join(clusters_dir, f"{word}.csv"), sep="\t", quoting=csv.QUOTE_NONE)
    word_clusters[CLUSTER_NUMBER_COLUMN] = word_clusters[CLUSTER_NUMBER_COLUMN].astype(int)

    
    this_word = word_uses.join(word_clusters.set_index("identifier"), on="identifier")
    #print(this_word.head())
    
    clusters_and_definitions = pd.read_csv(os.path.join(word_path, "cluster_gloss.tsv"), sep="\t")
    clusters_and_definitions["cluster"] = clusters_and_definitions.cluster.astype(int)
    clusters_and_definitions = clusters_and_definitions[clusters_and_definitions.cluster!=-1]
    if clusters_and_definitions.shape[0] > 1:
        
    
        definitions, contexts_list = [], []
        for row in clusters_and_definitions.iterrows():
            cluster_number, definition = row[1]
            
            
            this_cluster = this_word[this_word[CLUSTER_NUMBER_COLUMN] == cluster_number]
            
            contexts = []
    
            for row in this_cluster.iterrows():
                start, end = row[1]["indexes_target_token"].split(":")
                start, end = int(start), int(end)
                example = row[1][EXAMPLE_COLUMN]
                contexts.append(f"{example[:start]}<b>{example[start:end]}</b>{example[end:]}")

            if not contexts: 
                print(word)
                print(cluster_number)
            try:
                contexts_list.append('<br>'.join(set(random.choices(contexts, k=5))))
            except IndexError:
                print(contexts)
                raise IndexError
            definitions.append(definition)
        
        for definition in definitions:
            cluster_data = {}
            cluster_data["data"] = {"my_text": f"{word.upper()}: {definition}"}
            cluster_data["data"]["variants"] = [{"html": ctx} for ctx in contexts_list]
            label_data.append(cluster_data)
with open(os.path.expanduser(f"~/PycharmProjects/label-studio-{lang}.json"), "w") as f:
    json.dump(label_data, f)

Labeling interface

In [ ]:
# <View>
#   <Text name="query" value="$my_text"/>
#   <Header value="Select the best cluster of examples for this definition:"/>
#   <View style="display: flex">
#     <View>
#       <Choices name="selection" toName="query" required="true" choice="single-radio" value="$variants">
#       </Choices>
#     </View>
#   </View>
# </View>